In [4]:
# https://pypi.org/project/torch-geometric/
# pip install torch
# pip install torch-scatter -f https://data.pyg.org/whl/torch-1.13.0+cpu.html
# pip install torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cpu.html
# pip install torch-geometric


# https://towardsdatascience.com/louvains-algorithm-for-community-detection-in-python-95ff7f675306
import torch
from torch import Tensor
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from collections import Counter 

# 一、Data Inclusion
<br>
先將 training data include 進來，可以使用 print 看一下 train_data 長甚麼樣子，基本上就是以下這個 format <p>circleID: friend1 friend2 friend3 ... <p/>

總共有57種 features<br>

In [678]:
import os

# Include Training dataset
target_dir = os.getcwd()+"\Training"

data = []
for filename in os.listdir(target_dir):
    with open(os.path.join(target_dir, filename), 'r') as f:
        data += f.readlines()

train_data = []
for i in range(len(data)):
    train_data.append(data[i].split()[1:]) # circle* is seems not nesseary

print(f"number of trainin data: {len(train_data)}")
    
    
    
# Total is 57 kinds of features

with open("featureList.txt", 'r') as f:
    feature_list = f.readlines()

for i in range(len(feature_list)):
    feature_list[i] = feature_list[i].replace('\n', '')

print(f"number of features : {len(feature_list)}")


number of trainin data: 592
number of features : 57


# 二、Data Preprocessing

之後可以在這裡做一些 feature selection 之類的、或者視覺化<br>
有一些 features 是 NaN，可以做一些處理

舉例:   
26321.circles 和 25773.circles 都有circle 10分別是  
26321.circles -> circle10: 26379 26455 26482 26340 26409 26366 26390 26477 26415 26462 26402  
25773.circles -> circle10: 25787 25924 25947 25836 25817 25857 25902 25815 25821 25774 25961 ...
  
應該要將同樣 circle 的 user 加在一起，因為他們都是同一個 circle  
  
在沒這樣做之前，circle會重複，所以會誤以為是 592個，但實際上應該是455個才對

In [679]:
# 先找出重複的circle有幾個，再針對他們做合併，應該比較快，就不用一個一個還要重複掃
tes = []
for i in range(len(data)):
    tes.append(data[i].split(':')[0])

a = Counter(tes).most_common()[:46] # 有重複的總共46個
b = []
for i in range(len(a)):
    b.append(a[i][0])
print(f"Show the fist 5 elements: {b[:5]}")
print(f"Actually circle numbers: {len(Counter(tes))} not {len(tes)}")

Show the fist 5 elements: ['circle13', 'circle18', 'circle62', 'circle10', 'circle74']
Actually circle numbers: 455 not 592


In [680]:
temp_a = []

for target in b:
    temp = []
    for i in range(len(data)):
        circle_name = data[i].split(':')[:1][0]
        if circle_name == target:
            temp += data[i].split()[1:]

    temp_a.append(temp)

for i in range(len(data)):
    circle_name = data[i].split(':')[:1][0]
    
    if circle_name not in b: #如果 circle name 沒有重複的名單中，加入到list中
        temp_a.append(data[i].split()[1:])
        
print(f"Examine if the number is correct : {len(temp_a)}. The answer is yes, we eliminate repeated circles.")

Examine if the number is correct : 455. The answer is yes, we eliminate repeat cicles.


#### label每個node

In [695]:
# initialization every user's label

def label_data(x, circle): # x: number of dataset, circle: temp_a
    y_label = [0] * x #27520

    for i in range(len(circle)):
        for item in circle[i]:
            y_label[int(item)] = i+1
    return y_label

# 預想是說一個 user 應該會對應到一個 circle
# 但 0 特別多，代表有很多 user 沒有標到
# 下面 aaaa 是在算 train_data 中所有的 user，train_data代表將所有.circles的檔案合成一個list
# print(len(y_label)-Counter(y_label).most_common()[0][1])

aaaa = []
for i in train_data:
    aaaa+=i
#print(len(Counter(aaaa)))

In [739]:
y_label = label_data(len(df), temp_a)

#### Include 每個 Node 對應的 features，並且把它包成 DataFrame，但後續還需要做處理...<br>

In [740]:
# Include each user's features 
    
fea_dir = "features\\features.txt"
features = []

with open(fea_dir, 'r') as f:
    features += f.readlines()

a = [] 

for i in range(len(features)):
    # split each user features by space
    a.append(features[i].split()) 

final = []

# make a dict that key is feature and value is corresponeding value
for j in range(len(a)):
    b_col = []
    b_row = []
    for i in range(len(a[j][1:])):
        b_col.append(a[j][1:][i][::-1].split(';', 1)[1][::-1])
        b_row.append(a[j][1:][i].split(';')[-1])

    final.append(dict(zip(b_col, b_row)))
    
df = pd.DataFrame(final)
print(f"number of user:  {len(df)}")
#df # there are many NaN
df_new = df.fillna(-1)

df_new

number of user:  27520


,last_name,first_name,birthday,name,gender,locale,hometown;name,hometown;id,education;school;name,education;school;id,...,work;projects;from;name,work;projects;from;id,education;classes;description,work;from;name,work;from;id,political,religion,work;projects;end_date,work;projects;description,location
0,0,0,0,0,0,0,0,0,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,1,1,1,1,1,0,1,1,3,3,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,2,2,2,2,1,0,2,2,4,4,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,3,3,3,3,1,0,3,3,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,4,4,4,4,0,0,4,4,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27515,4416,3040,4794,27280,1,1,-1,-1,4174,22356,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
27516,17149,2944,-1,27281,0,0,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
27517,17150,470,-1,27282,1,1,133,133,395,405,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
27518,17151,3002,3531,27283,0,8,4299,4313,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [741]:
# 刪除資料集沒有circle的user
abc = df_new.copy()

# assign "label" as a feature in dataframe
abc = abc.assign(label = y_label)

# drop the rows if value in label is zero, and direct delete data
abc.drop(abc[abc.label==0].index, inplace=True)

In [790]:
def select_data(n): # select the dataframe based on its label frequency
    top_n = Counter(abc['label']).most_common()[:n]
    top_n_label = [top_n[i][0] for i in range(len(top_n))]
    
    options = top_n_label
    rslt_df = abc[abc['label'].isin(options)] # select features which is in top_n_label
    return rslt_df

In [818]:
rslt_df50 = select_data(50) # choose user whose belonging circle is top 50 in all circle (total 455)
rslt_df100 = select_data(100)
rslt_df150 = select_data(150)

In [827]:
rslt_df50['label']

244        2
245        2
252        2
257        2
268        2
        ... 
27512    368
27513    368
27516    368
27518    368
27519    368
Name: label, Length: 5960, dtype: int64

# 三、Model

Louvain Community Detection  
Surprise Community Detection  
Leiden Community Detection  
Walktrap Community Detection

In [297]:
from sklearn import tree
from sklearn.model_selection import KFold, cross_val_score


def decision_tree(df_new, y_label_new, n_splits = 10, shuffle=True):

    X, y = np.array(df_new), np.array(y_label_new) 

    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(X, y)
    
    k_folds = KFold(n_splits = n_splits, shuffle=True, random_state=True)

    scores = cross_val_score(clf, X, y, cv = k_folds, scoring='accuracy')
    
    print("Cross Validation Scores: ", scores)
    print("Average CV Score: {:.1%} ".format(scores.mean()))
    print("Number of CV Scores used in Average: ", len(scores))
    
    return clf

In [837]:
clf = decision_tree(rslt_df50[rslt_df50.columns[:-1]], rslt_df50['label'])

Cross Validation Scores:  [0.6409396  0.67785235 0.68624161 0.66778523 0.66610738 0.64932886
 0.65268456 0.66778523 0.64597315 0.67114094]
Average CV Score: 66.3% 
Number of CV Scores used in Average:  10


In [838]:
clf = decision_tree(rslt_df100[rslt_df100.columns[:-1]], rslt_df100['label'])

Cross Validation Scores:  [0.5370138  0.54203262 0.52823087 0.55834379 0.52195734 0.5621079
 0.55959849 0.55332497 0.54705144 0.55332497]
Average CV Score: 54.6% 
Number of CV Scores used in Average:  10


In [839]:
clf = decision_tree(rslt_df150[rslt_df150.columns[:-1]], rslt_df150['label'])

Cross Validation Scores:  [0.46847826 0.47173913 0.4673913  0.46847826 0.46195652 0.48695652
 0.47717391 0.46304348 0.46789989 0.47334059]
Average CV Score: 47.1% 
Number of CV Scores used in Average:  10


try GCN 

#### 合成 edge list

In [702]:
egonets = []
for filename in os.listdir(os.getcwd()+"\egonets"):
    with open(os.path.join(os.getcwd()+"\egonets", filename), 'r') as f:
        egonets += f.readlines()

In [703]:
pre = []

for i in range(len(egonets)):
    try:
        if int(egonets[i].split(":")[0]) in df_new.index: # 把沒有出現在df_new的user的egonet 去除掉
            pre.append(egonets[i])
    except:
        print(i)
        break

In [704]:
pre_final = []
for ego in pre:
    e1, es = ego.split(':')
    es = es.split()

    for e in es:
        pre_final.append((int(e1), int(e)))

In [661]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(32, 16)
        self.conv2 = GCNConv(16, 144)
        self.conv3 = GCNConv(144, 144)
        self.conv4 = GCNConv(144, 334)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = self.conv4(x, edge_index)

        return F.log_softmax(x, dim=1)

In [662]:
# 重整 label，讓原本可能是 0, 1, 2, 4, 9 變成按照順序 0, 1, 2, 3, 4
y_label_final = []

for i in range(len(y_label_new)):
    y_label_final.append(new.index(y_label_new[i]))

In [663]:
# make train mask
train_mask,  test_mask = [], []

def make_mask(num_of_training, df_x):
    for i in range(num_of_training):
        train_mask.append(True)
        test_mask.append(False)
        
    for i in range(len(df_x)-num_of_training):
        train_mask.append(False)
        test_mask.append(True)
        
    return torch.tensor(train_mask), torch.tensor(test_mask)

In [664]:
train_mask, test_mask = make_mask(9000, df_new)

In [673]:
from torch_geometric.data import Data

x = torch.tensor(np.array(df_new.values, dtype=float), dtype=torch.float) 
y = torch.tensor(y_label_new)
y = torch.tensor(y_label_final)
edge_index = torch.tensor(pre_final)

edge_index = edge_index[:12000]
data = Data(x=x,y=y,edge_index=edge_index.transpose(0,1), train_mask=train_mask, test_mask=test_mask)
data

Data(x=[11244, 32], edge_index=[2, 12000], y=[11244], train_mask=[11244], test_mask=[11244])

In [674]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
dataset = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(dataset)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [675]:
model.eval()
pred = model(dataset).argmax(dim=1)
correct = (pred[dataset.test_mask] == dataset.y[data.test_mask]).sum()
acc = int(correct) / int(dataset.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.0067


In [ ]:
edge_index